# Determine whether candidates for mQTL validation are supported by an eQTL

In [2]:
.libPaths("~/R/x86_64-redhat-linux-gnu-library/3.2/")
# config opts and libraries
options(repr.plot.width = 6)
options(repr.plot.height = 5)
library(ggplot2);
library(plyr);
library(dplyr);
library(reshape2);
library(LSD);
library(pheatmap);
library(parallel);
options(mc.cores = 24);
library(stringr);
library(RColorBrewer);
library(qtl);
library(funqtl);

In [3]:
# load eQTLs
load("/g/steinmetz/brooks/genphen/transcriptome/qtl/eQTL_08032016.rda")

mQTL candidates were selected with <a href="http://steinmetzlab.embl.de/shiny/mQTL/">mQTL Explorer</a> manually. Uploaded to an excel sheet. The excel sheet was exported as tab-delimted text

In [10]:
f = "/g/steinmetz/brooks/genphen/validation/mQTL_candidates_ANB.txt"
mQTL_candidates = read.delim(f, sep = "\t")
# to correct for re-running script
mQTL_candidates = mQTL_candidates %>% select(
    Metabolite, 
    Suggest.Selection,
    Detailed.Analysis,
    Bayesian.Confidence.Interval,
    FDR,
    Sys.Name,
    Name,
    STITCH,
    SNPs,
    InDels,
    Upstream,
    Downstream,
    Introns,
    Coding,
    High.Consequence,
    Moderate.Consequence,
    Low.Consequence,
    Chr,
    Start,
    End,
    Strand)

In [12]:
mQTL_eQTL_overlap = mQTL_candidates %>% rowwise() %>% do({
    thisgene = levels(.$Sys.Name)[.$Sys.Name]
    thischr = levels(.$Chr)[.$Chr]
    thisstart = .$Start
    thisend = .$End
    thisgeneqtls = eQTL_table %>% filter(gene == thisgene, seqnames == thischr)
    thisgeneqtls_all = eQTL_table %>% filter(gene == thisgene)
    if (dim(thisgeneqtls)[1]>0) {
        qtlranges = lapply(seq(1,dim(thisgeneqtls)[1]), function(i){
            seq(thisgeneqtls[i,"start"],thisgeneqtls[i,"end"],by = 1)
        })
        generange = seq(thisstart,thisend,by = 1)
        overlap = lapply(seq(1,length(qtlranges)), function(i){
            thisoverlap = length(intersect(generange, qtlranges[[i]]))
            if (thisoverlap > 0) {
                return(T)
            } else {
                return(F)
            }
        })
        if (T %in% unlist(overlap)) {
            cis.eqtl = T
        }
    } else {
        cis.eqtl = F
    }
    if (dim(thisgeneqtls_all)[1]>0) {
        any.eqtl = T
    } else {
        any.eqtl = F
    }
    return(data.frame(., cis.eqtl, any.eqtl))
}) 

In [14]:
head(mQTL_eQTL_overlap)

,Metabolite,Suggest.Selection,Detailed.Analysis,Bayesian.Confidence.Interval,FDR,Sys.Name,Name,STITCH,SNPs,InDels,Upstream,Downstream,Introns,Coding,High.Consequence,Moderate.Consequence,Low.Consequence,Chr,Start,End,Strand,cis.eqtl,any.eqtl
1,GLY_LEU_AKG_PYR_HSE_MET,NA,,0.9,0.05,YML089C,,NA,1,0,0,0,0,1,0,0,1,chrXIII,91041,91409,-,FALSE,FALSE
2,GLY_LEU_AKG_PYR_HSE_MET,NA,,0.9,0.05,YML090W,,NA,2,0,0,0,0,2,0,1,1,chrXIII,90744,91130,+,FALSE,FALSE
3,GLY_LEU_AKG_PYR_HSE_MET,NA,,0.9,0.05,YML091C,RPM2,NA,21,0,0,0,0,21,0,11,10,chrXIII,87123,90731,-,TRUE,TRUE
4,GLY_LEU_AKG_PYR_HSE_MET,NA,,0.9,0.05,YML092C,PRE8,NA,1,0,0,0,0,1,0,1,0,chrXIII,85987,86739,-,FALSE,FALSE
5,GLY_LEU_AKG_PYR_HSE_MET,NA,,0.9,0.05,YML093W,UTP14,NA,10,0,0,0,0,10,0,3,7,chrXIII,83090,85789,+,FALSE,FALSE
6,GLY_LEU_AKG_PYR_HSE_MET,NA,,0.9,0.05,YML094C-A,,NA,1,0,0,0,0,1,0,0,1,chrXIII,82219,82620,-,FALSE,FALSE


In [15]:
write.table(mQTL_eQTL_overlap, sep = "\t", file = f, quote = F, row.names = F)